In [1]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from skimage.data import imread
import matplotlib.pyplot as plt
from PIL import Image
import os

In [2]:
# ref: https://www.kaggle.com/paulorzp/run-length-encode-and-decode
def rle_decode(mask_rle, shape=(768, 768)):
    '''
    mask_rle: run-length as string formated (start length)
    shape: (height,width) of array to return 
    Returns numpy array, 1 - mask, 0 - background

    '''
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0]*shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    return img.reshape(shape).T  # Needed to align to RLE direction

In [3]:
masks = pd.read_csv('/home/ubuntu/input/train_ship_segmentations.csv')

In [14]:
!mkdir /home/ubuntu/input/train_mrcnn
# delete index later

ship_image_index = ~masks.EncodedPixels.isnull()
ship_image_ids = masks.ImageId[ship_image_index]
# remove duplicate
ship_image_ids = pd.Series.unique(ship_image_ids)
print('total image number is: '+ str(sum(ship_image_index)))
i=0
for ImageIdFull in ship_image_ids:
    i=i+1
    if (i%100 == 0):
        print(i)
    
    ImageId = ImageIdFull[:-4]
    img = imread('/home/ubuntu/input/train/' + ImageId + ".jpg")
    
    os.system("mkdir /home/ubuntu/input/train_mrcnn/" + ImageId)
    os.system("mkdir /home/ubuntu/input/train_mrcnn/" + ImageId+"/images")
    # may need to write a png
    Image.fromarray(img).save("/home/ubuntu/input/train_mrcnn/"+ ImageId+
             "/images/" + ImageId + ".png")
    
    os.system("mkdir /home/ubuntu/input/train_mrcnn/" + ImageId.replace(".", "_")+"/masks")
    img_masks = masks.loc[masks['ImageId'] == ImageIdFull, 'EncodedPixels'].tolist()
    # Take the individual ship masks and create a single mask array for all ships
    all_masks = np.zeros((768, 768))
    for i in range(len(img_masks)):
        mask = img_masks[i]
        one_mask = rle_decode(mask)
        one_mask[one_mask > 0] = 255
        im = Image.fromarray(one_mask)
        file_name = ImageId + "_" + str(i) + '.png'
        im.save("/home/ubuntu/input/train_mrcnn/"+ImageId+"/masks/"+file_name+ ".png")

total image number is: 56030


In [3]:
ship_detection = pd.read_csv('ship_detection.csv')
test_names = ship_detection.loc[ship_detection['p_ship'] > 0.5, ['id']]['id'].values.tolist()
test_names_nothing = ship_detection.loc[ship_detection['p_ship'] <= 0.5, ['id']]['id'].values.tolist()
len(test_names), len(test_names_nothing)
# test_names[0:10]

(3318, 12288)

100


In [18]:
!mkdir /home/ubuntu/input/test_mrcnn
i=0
for ImageIdFull in test_names:
    # print(ImageId)
    i=i+1
    if (i%100 == 0):
        print(i)
    ImageId = ImageIdFull[:-4]
    img = imread('/home/ubuntu/input/test_v2/' + ImageId + ".jpg")
    
    os.system("mkdir /home/ubuntu/input/test_mrcnn/" + ImageId)
    os.system("mkdir /home/ubuntu/input/test_mrcnn/" + ImageId+"/images")
    # may need to write a png
    Image.fromarray(img).save("/home/ubuntu/input/test_mrcnn/"+ ImageId+
             "/images/" + ImageId + ".png")

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300


'cp /home/ubuntu/input/train/0002756f7.jpg /home/ubuntu/input/train_mrcnn0002756f7_jpg'